In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/VNAV/
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data_example.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/VNAV


In [3]:
# using https://github.com/lyakaap/NetVLAD-pytorch implementation

import torch
import torch.nn as nn
from torch.autograd import Variable

from netvlad import NetVLAD
from netvlad import EmbedNet
from hard_triplet_loss import HardTripletLoss
from torchvision.models import resnet18


# Discard layers at the end of base network
encoder = resnet18(pretrained=True)
base_model = nn.Sequential(
    encoder.conv1,
    encoder.bn1,
    encoder.relu,
    encoder.maxpool,
    encoder.layer1,
    encoder.layer2,
    encoder.layer3,
    encoder.layer4,
)
dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)

# Define model for embedding
net_vlad = NetVLAD(num_clusters=32, dim=dim, alpha=1.0)
model = EmbedNet(base_model, net_vlad).cuda()

# Define loss
criterion = HardTripletLoss(margin=0.1).cuda()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s]


In [26]:

# This is just toy example. Typically, the number of samples in each classes are 4.
labels = torch.randint(0, 10, (40, )).long().cuda()
x = torch.rand(40, 3, 128, 128).cuda()
output = model(x)
# fw_output = model.forward(x)

triplet_loss = criterion(output, labels)

In [27]:
output

tensor([[-1.9619e-03, -3.8447e-04, -1.0705e-02,  ..., -1.1778e-02,
         -6.4315e-03, -6.2501e-03],
        [-2.3603e-03, -3.2227e-04, -1.0329e-02,  ..., -1.1645e-02,
         -5.1624e-03, -6.4710e-03],
        [-1.9879e-03, -9.0182e-06, -1.0040e-02,  ..., -1.1550e-02,
         -6.1690e-03, -6.1888e-03],
        ...,
        [-2.5051e-03, -5.5473e-04, -1.0517e-02,  ..., -1.1847e-02,
         -6.4067e-03, -6.3728e-03],
        [-2.0092e-03, -5.0645e-04, -1.0648e-02,  ..., -1.1548e-02,
         -6.0797e-03, -5.9868e-03],
        [-2.2812e-03, -2.5367e-04, -1.0423e-02,  ..., -1.1797e-02,
         -5.5962e-03, -6.1345e-03]], device='cuda:0', grad_fn=<DivBackward0>)

In [17]:
import numpy as np
print(np.all(torch.Tensor.numpy(fw_output.cpu() == output.cpu())))

True
